In [37]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

In [38]:
load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [ ]:
# define state
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    rating: int

In [ ]:
def create_outline(state: BlogState) -> BlogState:
    # fetch title
    title = state["title"]

    # create a prompt for llm
    prompt = f"Generate a detailed outline for a blog on the topic - {title}"

    # call the llm
    outline = model.invoke(prompt).content

    # update state
    state["outline"] = outline

    return state


def create_blog(state: BlogState) -> BlogState:
    # fetch title and outline
    title = state["title"]
    outline = state["outline"]

    # create a prompt for llm
    prompt = f"Write a detailed blog on the topic - {title} using the following outline \n {outline}"

    # call the llm
    content = model.invoke(prompt).content

    # update state
    state["content"] = content

    return state


def rate_blog(state: BlogState) -> BlogState:
    # fetch title and content
    title = state["title"]
    content = state["content"]

    # create a prompt for llm
    prompt = f"Based on the topic - {title} evaluate the following blog content \n {content} \n and give a rate out of 10 it should be a number only give the number rating dont include any other strings"

    # call the llm
    rating = model.invoke(prompt).content

    # update state
    state["rating"] = rating

    return state

In [ ]:
# define graph
graph = StateGraph(BlogState)

# add nodes
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)
graph.add_node("rate_blog", rate_blog)

# add edges
graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", "rate_blog")
graph.add_edge("rate_blog", END)

# compile graph
workflow = graph.compile()

In [ ]:
# execute the graph workflow
initial_state = {"title": "Rise of AI in india"}
final_state = workflow.invoke(initial_state)

print(final_state)

{'title': 'Rise of AI in india', 'outline': "## Blog Outline: The Rise of AI in India\n\n**I. Introduction (approx. 100 words)**\n\n*   Hook: Start with a compelling statistic or anecdote showcasing AI's growing presence in India (e.g., investment figures, a successful AI application).\n*   Briefly define AI and its various applications.\n*   Thesis statement:  Highlight the rapid growth of AI in India, outlining the key factors driving this growth and its potential impact on the nation's economy and society.\n*   Brief overview of the blog's structure.\n\n\n**II. Drivers of AI Growth in India (approx. 300 words)**\n\n*   **A. Government Initiatives:**\n    *   Discuss policies like the National AI Strategy, Digital India, and initiatives promoting AI education and research.\n    *   Highlight government funding and support for AI startups and research institutions.\n    *   Mention any specific government projects utilizing AI (e.g., in healthcare, agriculture).\n*   **B. Growing Tale

In [43]:
print(final_state["rating"])

9
